In [40]:
import pandas as pd
import numpy as np
from csv import reader
from numpy import linalg as la
from sys import exit
from haversine import haversine
import math
from operator import itemgetter
from numpy import dot
from numpy.linalg import norm
#시각화 도구
import folium
from folium.features import DivIcon
import random
import matplotlib.pyplot as plt
from shutil import copyfile
import os
import random
# 전역 변수 설정
FILE_SAVE_DIR = './Data_set_0826/' #있다면 추가 해주기 (상대경로)
line_color_list = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
                    'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white',
                    'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray'] 
# Global Variable Setting

# Nicname = '5CAC8432AE4EC54B'
# Huzzi = '44754E5A077F32B4'
# oizi = '2DDB3706DE4F7B45'
kyounghwan = '3963650B38CAF112'
hyeryeong = '4CCF4C0B0F953026'
pyeongju = '8A57AE95DD4629A7'
joongho = '9E7F4CB4F4A6A6D4'
yujin = 'F9888CC2D3053565'
member_list = ['3963650B38CAF112','4CCF4C0B0F953026','8A57AE95DD4629A7','9E7F4CB4F4A6A6D4','F9888CC2D3053565']
PATIENT_UID = kyounghwan # 감염자 UID
TARGET_UID =  hyeryeong # 조사 대상자 UID

# Map Setting
Raw_map = folium.Map(location = [37.5505938572,127.074236903], zoom_start =100) # 세종대학교 중심 조사
Filter_map = folium.Map(location = [37.5505938572,127.074236903], zoom_start =100) # 세종대학교 중심 조사
mass_map = folium.Map(location = [37.5505938572,127.074236903], zoom_start =100) # 세종대학교 중심 조사
compare_map = folium.Map(location = [37.5505938572,127.074236903], zoom_start =100) # 세종대학교 중심 조사

# dirCnt = 10
# for dir in range(dirCnt):
#     os.makedirs('./noise_injection/'+str(dir))
#     for member in member_list:
#         copyfile(FILE_SAVE_DIR + member +"_gps"+".csv",'./noise_injection/'+str(dir)+'/'+ member +"_gps"+".csv")
def load_GPS_set(UID, scnario, file_path): # Data load
    try:
        if scnario == 1:
            data_set = pd.read_csv(file_path + UID +"_gps"+".csv", sep=",", dtype='unicode')
            data_set['TIME STAMP'] = pd.to_datetime(data_set['loggingTime']).dt.strftime('%Y-%m-%d %H:%M:%s')
            is_scnario = data_set['scinario'] == '1'
            data_set = data_set[is_scnario]
            re_data_set = data_set[["TIME STAMP","uid","latitude","longitude","provider"]]
            return re_data_set
        elif scnario == 2:
            data_set = pd.read_csv(file_path + UID +"_gps"+".csv", sep=",", dtype='unicode')
            data_set['TIME STAMP'] = pd.to_datetime(data_set['loggingTime']).dt.strftime('%Y-%m-%d %H:%M:%s')
            is_scnario = data_set['scinario'] == '2'
            data_set = data_set[is_scnario]
            re_data_set = data_set[["TIME STAMP","uid","latitude","longitude","provider"]]
            re_data_set.reset_index(inplace=True, drop=True)
            return re_data_set   
    except FileNotFoundError as e:
        pass


In [41]:
haversine([37.54758427,127.0735179],[37.54758927,127.0735679], unit = 'm') #0.00005 / 0.00005
float(random.random()/10000)

4.3678238324081286e-05

In [42]:
#Noise Injection - 위도 경도 0.0001 ~ 0.00001 렌덤 노이즈 부여 +-0.0001 => 10m +- 0.00001 => 1m
dirCnt= 10
for dir in range(dirCnt):
    for member in member_list:
        data_set = pd.read_csv('./noise_injection/'+str(dir)+ "/"+ member +"_gps"+".csv", sep=",", dtype='unicode')
        data_set['TIME STAMP'] = pd.to_datetime(data_set['loggingTime']).dt.strftime('%Y-%m-%d %H:%M:%s')
        for count in range(len(data_set.index)):
            data_set.loc[count,'latitude'] = float(data_set.loc[count,'latitude']) +  random.uniform(-0.00005, 0.00005)
            data_set.loc[count,'longitude'] = float(data_set.loc[count,'longitude']) +  random.uniform(0.00005, 0.00001)
        data_set.to_csv('./noise_injection/'+str(dir)+ "/"+ member +"_gps"+".csv")

In [43]:
################### RAW DATA FUNC ######################
def extract_raw_data(UID, scnario,file_path):
    Dataframe = load_GPS_set(UID, scnario,file_path)
    marker_list = []
    for count in range(len(Dataframe.index)):
        marker_point_lati = float(Dataframe.loc[count,'latitude'])
        marker_point_long = float(Dataframe.loc[count,'longitude'])
        marker_point_time = Dataframe.loc[count,'TIME STAMP']
        marker_point = [marker_point_time, marker_point_lati, marker_point_long]
        marker_list.append(marker_point)
    return marker_list
    
kyounghwan_raw = extract_raw_data('3963650B38CAF112', 1, FILE_SAVE_DIR)
kyounghwan_noise = extract_raw_data('3963650B38CAF112', 1, './noise_injection/0/')
# hyeryeong_raw = extract_raw_data('4CCF4C0B0F953026', 1)
# pyeongju_raw = extract_raw_data('8A57AE95DD4629A7', 1)
# joongho_raw = extract_raw_data('9E7F4CB4F4A6A6D4', 1)
# yujin_raw = extract_raw_data('F9888CC2D3053565', 1)


In [ ]:
# Random Split Windows


In [44]:
    
################### ABOUT MAP DROWING FUNC #####################
def draw_Polyline_map(map, marker_list, draw_color):
    replace_list = []
    for marker_point in marker_list:
        replace_list.append(marker_point[1:3])
    folium.PolyLine(locations=replace_list,tooltip='Polyline',color=draw_color).add_to(map)

def draw_index_map(map, marker_list):
    count = len(marker_list)
    for marker_point in marker_list:
        folium.Marker(marker_point[1:3], icon=DivIcon(icon_size=(10,3),icon_anchor=(7,20),
        html='<div style="font-size: 10pt; color : black">'+str(count)+'</div>',
        )).add_to(map)
        count = count - 1

def draw_circle_map(map, marker_list,draw_radius,draw_color):
    for marker_point in marker_list:
        folium.CircleMarker(location=marker_point[1:3], radius=draw_radius, color= draw_color).add_to(map)
        
def draw_arrow_map(map, start_list, end_list, line_color, dot_color):
    for start_point in start_list:
        for end_point in end_list:
            if(start_point[0]==end_point[0]):
                try:
                    rot_x = end_point[1] - start_point[1]
                    rot_y = end_point[2] - start_point[2]
                    rot = math.acos(haversine([start_point[1],0],[end_point[1],0])/haversine(start_point[1:3],end_point[1:3]))
                    # folium.CircleMarker(location=end_point[1:3], radius=1, color= dot_color).add_to(map)
                    folium.PolyLine(locations=[start_point[1:3],end_point[1:3]],tooltip='Polyline',color=line_color).add_to(map)
                    folium.CircleMarker(location=start_point[1:3], radius=3, color= dot_color).add_to(map)
                    folium.RegularPolygonMarker(end_point[1:3], fill_color='blue', number_of_sides=3, radius=3, rotation=rot).add_to(map)
                except IndexError :
                    break
                except ZeroDivisionError:
                    break
                

In [45]:
draw_Polyline_map(Filter_map,kyounghwan_raw,"red")
draw_Polyline_map(Filter_map,kyounghwan_noise,"blue")
Filter_map